# Scientific Computing & Numerical Methods

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** Days 253-280
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
# ── Imports & matplotlib configuration ──────────────────────────────
import numpy as np
import scipy
from scipy import linalg, integrate, optimize, sparse
from scipy.sparse.linalg import eigsh, spsolve, cg
from scipy.integrate import solve_ivp, quad, dblquad
import matplotlib.pyplot as plt
from matplotlib import cm
import time

%matplotlib inline

# Publication-quality defaults
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 15,
    'legend.fontsize': 11,
    'lines.linewidth': 2,
    'figure.dpi': 100,
})

print(f"NumPy {np.__version__}  |  SciPy {scipy.__version__}")
print("All imports successful — ready for scientific computing.")

## 1. NumPy Advanced: Broadcasting, Vectorization & Memory Layout

NumPy's power comes from *vectorized operations* that run in compiled C/Fortran
rather than Python loops.  Three critical concepts:

| Concept | Description |
|---------|-------------|
| **Broadcasting** | Rules that let arrays of different shapes combine element-wise |
| **Vectorization** | Replacing Python loops with array operations |
| **Memory layout** | Row-major (C) vs column-major (Fortran) affects cache performance |

**Broadcasting rule:** Two dimensions are compatible when they are equal or one
of them is 1.  Dimensions are compared from the trailing (rightmost) axis.

$$\text{Shape }(3,1) + \text{Shape }(1,4) \rightarrow \text{Shape }(3,4)$$

### Quantum Connection
Tensor products in quantum mechanics follow similar broadcasting ideas:
if $|\psi\rangle$ has dimension $d_1$ and $|\phi\rangle$ has dimension $d_2$,
the product state lives in $d_1 \times d_2$ dimensions.

In [ ]:
# ── Broadcasting demonstration ──────────────────────────────────────
# Create a 2D Gaussian via broadcasting (no loops!)
x = np.linspace(-3, 3, 200)
y = np.linspace(-3, 3, 200)

# x is (200,) → reshape to (200, 1); y stays (200,)
# Broadcasting produces (200, 200)
X = x[:, np.newaxis]
Y = y[np.newaxis, :]

# 2D Gaussian with different widths
sigma_x, sigma_y = 1.0, 1.5
Z = np.exp(-X**2 / (2 * sigma_x**2) - Y**2 / (2 * sigma_y**2))

print(f"x shape: {x.shape}")
print(f"X (after newaxis) shape: {X.shape}")
print(f"Y (after newaxis) shape: {Y.shape}")
print(f"Z (broadcasted result) shape: {Z.shape}")
print(f"Z max = {Z.max():.4f}, Z min = {Z.min():.6f}")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Contour plot
c = axes[0].contourf(x, y, Z.T, levels=30, cmap='viridis')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('2D Gaussian via Broadcasting')
axes[0].set_aspect('equal')
plt.colorbar(c, ax=axes[0], label='Amplitude')

# Surface plot of the same data
ax3d = fig.add_subplot(122, projection='3d')
axes[1].remove()  # remove the flat axes[1]
Xm, Ym = np.meshgrid(x[::5], y[::5])
ax3d.plot_surface(Xm, Ym, Z[::5, ::5].T, cmap='viridis', alpha=0.9)
ax3d.set_xlabel('x')
ax3d.set_ylabel('y')
ax3d.set_zlabel('Amplitude')
ax3d.set_title('Surface Plot')

plt.tight_layout()
plt.show()

In [ ]:
# ── Vectorization speed comparison ──────────────────────────────────
N = 500_000

# Python loop version
def loop_norm(arr):
    result = 0.0
    for val in arr:
        result += val * val
    return np.sqrt(result)

# Vectorized version
def vec_norm(arr):
    return np.sqrt(np.sum(arr**2))

data = np.random.randn(N)

t0 = time.perf_counter()
r1 = loop_norm(data)
t_loop = time.perf_counter() - t0

t0 = time.perf_counter()
r2 = vec_norm(data)
t_vec = time.perf_counter() - t0

t0 = time.perf_counter()
r3 = np.linalg.norm(data)
t_np = time.perf_counter() - t0

print(f"{'Method':<20} {'Time (ms)':>10} {'Result':>14}")
print("-" * 46)
print(f"{'Python loop':<20} {t_loop*1000:>10.2f} {r1:>14.8f}")
print(f"{'Vectorized':<20} {t_vec*1000:>10.2f} {r2:>14.8f}")
print(f"{'np.linalg.norm':<20} {t_np*1000:>10.2f} {r3:>14.8f}")
print(f"\nSpeedup (vectorized vs loop): {t_loop/t_vec:.0f}x")
print(f"Speedup (np.linalg vs loop):  {t_loop/t_np:.0f}x")

In [ ]:
# ── Memory layout: C-order vs Fortran-order ─────────────────────────
N = 2000
A_c = np.random.randn(N, N)                      # C-order (row-major)
A_f = np.asfortranarray(A_c)                      # Fortran-order (col-major)

print(f"C-order flags: C_CONTIGUOUS={A_c.flags['C_CONTIGUOUS']}, "
      f"F_CONTIGUOUS={A_c.flags['F_CONTIGUOUS']}")
print(f"F-order flags: C_CONTIGUOUS={A_f.flags['C_CONTIGUOUS']}, "
      f"F_CONTIGUOUS={A_f.flags['F_CONTIGUOUS']}")

# Row-sum: access by rows (good for C-order)
t0 = time.perf_counter()
_ = A_c.sum(axis=1)
t_row_c = time.perf_counter() - t0

t0 = time.perf_counter()
_ = A_f.sum(axis=1)
t_row_f = time.perf_counter() - t0

# Column-sum: access by columns (good for F-order)
t0 = time.perf_counter()
_ = A_c.sum(axis=0)
t_col_c = time.perf_counter() - t0

t0 = time.perf_counter()
_ = A_f.sum(axis=0)
t_col_f = time.perf_counter() - t0

print(f"\n{'Operation':<20} {'C-order (ms)':>14} {'F-order (ms)':>14}")
print("-" * 50)
print(f"{'Row sum':<20} {t_row_c*1000:>14.3f} {t_row_f*1000:>14.3f}")
print(f"{'Column sum':<20} {t_col_c*1000:>14.3f} {t_col_f*1000:>14.3f}")
print("\nCache-friendly access patterns give measurable speedups.")

## 2. SciPy Linear Algebra: Sparse Matrices & Iterative Solvers

Many physics problems produce **sparse matrices** — matrices where most entries
are zero.  Storing only the nonzero elements saves enormous memory and enables
specialized fast algorithms.

**Key sparse formats in SciPy:**

| Format | Best for |
|--------|----------|
| `csr_matrix` | Row slicing, matrix-vector products |
| `csc_matrix` | Column slicing, sparse direct solvers |
| `coo_matrix` | Construction from triplets |
| `lil_matrix` | Incremental construction |

### Quantum Connection
The Hamiltonian of a quantum system with $n$ qubits lives in a $2^n \times 2^n$
Hilbert space, but many physically relevant Hamiltonians (e.g., nearest-neighbor
interactions) are extremely sparse.  Sparse eigensolvers let us find the ground
state energy of systems far beyond what dense methods allow.

In [ ]:
# ── Sparse matrix construction: 1D tight-binding Hamiltonian ────────
# H = -t Σ (|i><i+1| + |i+1><i|) + V(i)|i><i|
# This is the discrete Schrödinger equation on a lattice

N_sites = 500       # number of lattice sites
t_hop = 1.0         # hopping parameter
V0 = 0.5            # potential strength

# Build the sparse Hamiltonian
diag_main = V0 * np.cos(2 * np.pi * np.arange(N_sites) / N_sites)  # cosine potential
diag_off = -t_hop * np.ones(N_sites - 1)

H_sparse = sparse.diags(
    [diag_off, diag_main, diag_off],
    offsets=[-1, 0, 1],
    shape=(N_sites, N_sites),
    format='csr'
)

# Compare memory usage
H_dense = H_sparse.toarray()
sparse_bytes = H_sparse.data.nbytes + H_sparse.indices.nbytes + H_sparse.indptr.nbytes
dense_bytes = H_dense.nbytes

print(f"Matrix size: {N_sites} x {N_sites}")
print(f"Nonzero elements: {H_sparse.nnz} / {N_sites**2} "
      f"({100*H_sparse.nnz/N_sites**2:.2f}%)")
print(f"Dense memory:  {dense_bytes / 1024:.1f} KB")
print(f"Sparse memory: {sparse_bytes / 1024:.1f} KB")
print(f"Compression ratio: {dense_bytes / sparse_bytes:.1f}x")

# Find lowest 6 eigenvalues using sparse eigensolver
n_eig = 6
eigenvalues, eigenvectors = eigsh(H_sparse, k=n_eig, which='SA')

print(f"\nLowest {n_eig} eigenvalues:")
for i, ev in enumerate(eigenvalues):
    print(f"  E_{i} = {ev:+.6f}")

# Plot eigenstates
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
x = np.arange(N_sites)
for i, ax in enumerate(axes.flat):
    ax.plot(x, eigenvectors[:, i], color=f'C{i}', linewidth=1.5)
    ax.fill_between(x, eigenvectors[:, i], alpha=0.3, color=f'C{i}')
    ax.set_title(f'$\\psi_{i}$,  $E_{i}$ = {eigenvalues[i]:.4f}')
    ax.set_xlabel('Site index')
    ax.set_ylabel('Amplitude')
    ax.axhline(0, color='gray', linewidth=0.5)

plt.suptitle('Tight-Binding Model: Lowest 6 Eigenstates', fontsize=16)
plt.tight_layout()
plt.show()

## 3. Numerical Integration: Quadrature & Monte Carlo

Exact analytical integrals are rare in physics.  Numerical integration is
essential for computing expectation values, normalization constants, and
transition amplitudes.

### Methods Compared

| Method | Dimension | Convergence | Best for |
|--------|-----------|-------------|----------|
| `quad` (adaptive Gauss) | 1D | $O(h^{2n+1})$ | Smooth 1D integrands |
| `dblquad` | 2D | adaptive | Low-dimensional integrals |
| **Monte Carlo** | Any $d$ | $O(1/\sqrt{N})$ | High-dimensional integrals |

### Key formula
The Monte Carlo estimate of $\int_\Omega f(\mathbf{x})\,d\mathbf{x}$ over
volume $V$ using $N$ random samples:

$$I \approx \frac{V}{N} \sum_{i=1}^{N} f(\mathbf{x}_i), \quad
\sigma_I \approx \frac{V}{\sqrt{N}} \sqrt{\langle f^2 \rangle - \langle f \rangle^2}$$

### Quantum Connection
Quantum Monte Carlo methods use stochastic sampling to solve the many-body
Schrödinger equation for systems with dozens or hundreds of electrons.

In [ ]:
# ── Numerical integration comparison ────────────────────────────────
# Integral: ∫₀^∞ x² e^{-x} dx = Γ(3) = 2! = 2

# 1) SciPy quad (adaptive Gaussian quadrature)
result_quad, error_quad = quad(lambda x: x**2 * np.exp(-x), 0, np.inf)
print(f"quad result:   {result_quad:.10f} ± {error_quad:.2e}")
print(f"Exact answer:  {2.0:.10f}")

# 2) Double integral: ∫∫ e^{-(x²+y²)} dx dy over [-∞,∞]² = π
result_dbl, error_dbl = dblquad(
    lambda y, x: np.exp(-(x**2 + y**2)),
    -5, 5,     # x limits (approximate ∞)
    -5, 5      # y limits
)
print(f"\ndblquad result: {result_dbl:.10f} ± {error_dbl:.2e}")
print(f"Exact (π):      {np.pi:.10f}")

# 3) Monte Carlo integration of the same 2D Gaussian
rng = np.random.default_rng(42)
N_samples_list = [100, 1000, 10_000, 100_000, 1_000_000]
mc_results = []
mc_errors = []

for N_mc in N_samples_list:
    # Sample uniformly in [-5, 5] x [-5, 5], volume = 100
    xy = rng.uniform(-5, 5, size=(N_mc, 2))
    f_vals = np.exp(-(xy[:, 0]**2 + xy[:, 1]**2))
    volume = 10.0 * 10.0  # = 100
    estimate = volume * np.mean(f_vals)
    std_err = volume * np.std(f_vals) / np.sqrt(N_mc)
    mc_results.append(estimate)
    mc_errors.append(std_err)

print(f"\n{'N samples':>12} {'MC estimate':>14} {'Error':>12} {'Rel. error':>12}")
print("-" * 54)
for N_mc, est, err in zip(N_samples_list, mc_results, mc_errors):
    print(f"{N_mc:>12,} {est:>14.8f} {err:>12.6f} {abs(est-np.pi)/np.pi:>12.2e}")

# Plot convergence
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(N_samples_list, [abs(r - np.pi) for r in mc_results], 'o-',
          label='MC absolute error', markersize=8)
ax.loglog(N_samples_list, mc_errors, 's--', label='MC standard error', markersize=8)
ax.loglog(N_samples_list, 5 / np.sqrt(N_samples_list), 'k:',
          label=r'$O(1/\sqrt{N})$ reference')
ax.set_xlabel('Number of samples')
ax.set_ylabel('Error')
ax.set_title('Monte Carlo Convergence: $\\int e^{-(x^2+y^2)} \\, dA = \\pi$')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. ODE Solvers: Euler, RK4, and `solve_ivp`

Ordinary differential equations appear throughout physics — from classical
trajectories to the time-dependent Schrödinger equation.

### Methods

**Forward Euler** (1st order):
$$y_{n+1} = y_n + h\,f(t_n, y_n), \quad \text{error} = O(h)$$

**Runge-Kutta 4** (4th order):
$$y_{n+1} = y_n + \frac{h}{6}(k_1 + 2k_2 + 2k_3 + k_4), \quad \text{error} = O(h^4)$$

**SciPy `solve_ivp`**: Adaptive step-size control using RK45, RK23, DOP853,
Radau (implicit), and BDF methods.

### Quantum Connection
The time-dependent Schrödinger equation $i\hbar \frac{\partial}{\partial t}|\psi\rangle = \hat{H}|\psi\rangle$
is a first-order ODE in time, often solved with these same numerical methods.

In [ ]:
# ── ODE solver comparison: harmonic oscillator ──────────────────────
# ẍ + ω²x = 0  →  y = [x, v],  dy/dt = [v, -ω²x]
omega = 2 * np.pi  # frequency

def harmonic_rhs(t, y):
    return [y[1], -omega**2 * y[0]]

# Exact solution: x(t) = cos(ωt), v(t) = -ω sin(ωt)
y0 = [1.0, 0.0]
t_span = (0, 5)

# Forward Euler implementation
def euler_solve(f, t_span, y0, dt):
    t_arr = np.arange(t_span[0], t_span[1] + dt, dt)
    y_arr = np.zeros((len(t_arr), len(y0)))
    y_arr[0] = y0
    for i in range(len(t_arr) - 1):
        dydt = f(t_arr[i], y_arr[i])
        y_arr[i+1] = y_arr[i] + dt * np.array(dydt)
    return t_arr, y_arr

# Classical RK4 implementation
def rk4_solve(f, t_span, y0, dt):
    t_arr = np.arange(t_span[0], t_span[1] + dt, dt)
    y_arr = np.zeros((len(t_arr), len(y0)))
    y_arr[0] = y0
    for i in range(len(t_arr) - 1):
        t, y = t_arr[i], y_arr[i]
        k1 = np.array(f(t, y))
        k2 = np.array(f(t + dt/2, y + dt/2 * k1))
        k3 = np.array(f(t + dt/2, y + dt/2 * k2))
        k4 = np.array(f(t + dt, y + dt * k3))
        y_arr[i+1] = y + (dt / 6) * (k1 + 2*k2 + 2*k3 + k4)
    return t_arr, y_arr

dt = 0.01

# Solve with all three methods
t0 = time.perf_counter()
t_e, y_e = euler_solve(harmonic_rhs, t_span, y0, dt)
time_euler = time.perf_counter() - t0

t0 = time.perf_counter()
t_r, y_r = rk4_solve(harmonic_rhs, t_span, y0, dt)
time_rk4 = time.perf_counter() - t0

t0 = time.perf_counter()
sol = solve_ivp(harmonic_rhs, t_span, y0, method='RK45',
                t_eval=np.arange(t_span[0], t_span[1], dt),
                rtol=1e-10, atol=1e-12)
time_ivp = time.perf_counter() - t0

# Exact solution
t_exact = np.linspace(0, 5, 1000)
x_exact = np.cos(omega * t_exact)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Trajectories
axes[0].plot(t_exact, x_exact, 'k-', label='Exact', linewidth=2.5)
axes[0].plot(t_e, y_e[:, 0], '--', label=f'Euler (dt={dt})', alpha=0.8)
axes[0].plot(t_r, y_r[:, 0], '-.', label=f'RK4 (dt={dt})', alpha=0.8)
axes[0].plot(sol.t, sol.y[0], ':', label='solve_ivp (RK45)', linewidth=2)
axes[0].set_xlabel('Time')
axes[0].set_ylabel('x(t)')
axes[0].set_title('Harmonic Oscillator Solutions')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Errors
err_euler = np.abs(y_e[:, 0] - np.cos(omega * t_e))
err_rk4 = np.abs(y_r[:, 0] - np.cos(omega * t_r))
err_ivp = np.abs(sol.y[0] - np.cos(omega * sol.t))

axes[1].semilogy(t_e, err_euler + 1e-16, label='Euler')
axes[1].semilogy(t_r, err_rk4 + 1e-16, label='RK4')
axes[1].semilogy(sol.t, err_ivp + 1e-16, label='solve_ivp (RK45)')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('|error|')
axes[1].set_title('Absolute Error Comparison')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n{'Method':<18} {'Time (ms)':>10} {'Max error':>14} {'Final error':>14}")
print("-" * 60)
print(f"{'Euler':<18} {time_euler*1000:>10.2f} {err_euler.max():>14.2e} {err_euler[-1]:>14.2e}")
print(f"{'RK4':<18} {time_rk4*1000:>10.2f} {err_rk4.max():>14.2e} {err_rk4[-1]:>14.2e}")
print(f"{'solve_ivp RK45':<18} {time_ivp*1000:>10.2f} {err_ivp.max():>14.2e} {err_ivp[-1]:>14.2e}")

## 5. Optimization: Minimization, Root Finding & Curve Fitting

Optimization is ubiquitous in physics:
- Variational methods minimize $\langle \psi | \hat{H} | \psi \rangle$
- Least-squares fitting extracts physical parameters from experimental data
- Root finding locates energy eigenvalues from transcendental equations

### Key SciPy tools

| Function | Purpose |
|----------|---------|
| `optimize.minimize` | General-purpose minimization (BFGS, Nelder-Mead, etc.) |
| `optimize.root_scalar` | 1D root finding (Brent, bisection, Newton) |
| `optimize.curve_fit` | Nonlinear least-squares fitting |

### Quantum Connection
The **variational principle** states that for any trial wavefunction $|\psi_T\rangle$:

$$E_0 \leq \frac{\langle \psi_T | \hat{H} | \psi_T \rangle}{\langle \psi_T | \psi_T \rangle}$$

Minimizing this expression over parameters yields an upper bound on the ground
state energy — this is the foundation of variational quantum eigensolvers (VQE).

In [ ]:
# ── Optimization demonstrations ─────────────────────────────────────

# 1) Variational method: find ground state of quartic potential V = x⁴
#    Trial function: ψ(x) = (α/π)^{1/4} exp(-αx²/2)
#    ⟨H⟩ = α/4 + 3/(4α²)  (analytically derived)

def energy_quartic(alpha):
    """Energy expectation value for Gaussian trial in quartic potential."""
    return alpha / 4.0 + 3.0 / (4.0 * alpha**2)

alpha_range = np.linspace(0.3, 5, 200)
E_range = [energy_quartic(a) for a in alpha_range]

# Minimize
from scipy.optimize import minimize_scalar, minimize, curve_fit, root_scalar

result = minimize_scalar(energy_quartic, bounds=(0.1, 10), method='bounded')
alpha_opt = result.x
E_opt = result.fun

print("=== Variational Method: Quartic Oscillator ===")
print(f"Optimal α = {alpha_opt:.6f}")
print(f"Variational energy = {E_opt:.6f}")
print(f"Analytical optimum: α = {(3/2)**(1/3):.6f}, "
      f"E = {3/(4*(3/2)**(2/3)) + (3/2)**(1/3)/4:.6f}")

# 2) Root finding: transcendental equation for finite square well
#    z tan(z) = √(z₀² - z²)  where z₀ = √(2mVa²/ℏ²)
z0 = 8.0  # well depth parameter

def transcendental(z):
    if z >= z0:
        return float('inf')
    return z * np.tan(z) - np.sqrt(z0**2 - z**2)

# Find roots in intervals where tan(z) > 0
roots = []
for n in range(int(z0 / np.pi) + 1):
    lo = n * np.pi + 0.01
    hi = (n + 0.5) * np.pi - 0.01
    if lo < z0:
        try:
            sol = root_scalar(transcendental, bracket=[lo, min(hi, z0 - 0.01)])
            roots.append(sol.root)
        except ValueError:
            pass

print(f"\n=== Finite Square Well (z₀ = {z0}) ===")
print("Bound state z values (even parity):")
for i, r in enumerate(roots):
    E_ratio = r**2 / z0**2
    print(f"  n={i}: z = {r:.6f}, E/V₀ = {E_ratio:.6f}")

# 3) Curve fitting: noisy exponential decay
rng = np.random.default_rng(42)
t_data = np.linspace(0, 5, 50)
true_params = (3.0, 1.5, 0.5)  # A, γ, offset
y_data = true_params[0] * np.exp(-true_params[1] * t_data) + true_params[2]
y_noisy = y_data + 0.15 * rng.normal(size=len(t_data))

def exp_decay(t, A, gamma, offset):
    return A * np.exp(-gamma * t) + offset

popt, pcov = curve_fit(exp_decay, t_data, y_noisy, p0=[2, 1, 0])
perr = np.sqrt(np.diag(pcov))

print(f"\n=== Curve Fitting: Exponential Decay ===")
print(f"{'Param':<8} {'True':>8} {'Fitted':>10} {'±Error':>10}")
print("-" * 38)
for name, true, fit, err in zip(['A', 'γ', 'offset'], true_params, popt, perr):
    print(f"{name:<8} {true:>8.3f} {fit:>10.4f} {err:>10.4f}")

# Combined plot
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Variational energy
axes[0].plot(alpha_range, E_range, 'b-', linewidth=2)
axes[0].axvline(alpha_opt, color='r', linestyle='--', label=f'α* = {alpha_opt:.3f}')
axes[0].axhline(E_opt, color='r', linestyle=':', alpha=0.5)
axes[0].set_xlabel(r'$\alpha$')
axes[0].set_ylabel(r'$\langle E \rangle$')
axes[0].set_title('Variational Energy: Quartic Potential')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Transcendental equation
z_plot = np.linspace(0.01, z0 - 0.01, 500)
lhs = z_plot * np.tan(z_plot)
rhs = np.sqrt(z0**2 - z_plot**2)
lhs_clipped = np.where(np.abs(lhs) < 50, lhs, np.nan)
axes[1].plot(z_plot, lhs_clipped, 'b-', label=r'$z \tan(z)$')
axes[1].plot(z_plot, rhs, 'r-', label=r'$\sqrt{z_0^2 - z^2}$')
for r in roots:
    axes[1].axvline(r, color='green', linestyle='--', alpha=0.6)
axes[1].set_xlim(0, z0)
axes[1].set_ylim(-5, 30)
axes[1].set_xlabel('z')
axes[1].set_title('Finite Square Well: Root Finding')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Curve fitting
axes[2].scatter(t_data, y_noisy, s=30, alpha=0.7, label='Noisy data')
t_fine = np.linspace(0, 5, 200)
axes[2].plot(t_fine, exp_decay(t_fine, *true_params), 'g--',
             label='True curve', linewidth=2)
axes[2].plot(t_fine, exp_decay(t_fine, *popt), 'r-',
             label='Fitted curve', linewidth=2)
axes[2].set_xlabel('Time')
axes[2].set_ylabel('Signal')
axes[2].set_title('Nonlinear Curve Fitting')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Eigenvalue Problems for Large Systems

In quantum mechanics, finding eigenvalues means finding energy levels:

$$\hat{H}|\psi_n\rangle = E_n|\psi_n\rangle$$

For large systems ($N > 10^4$), storing the full matrix is impractical.
**Sparse eigensolvers** (Lanczos/Arnoldi algorithms via `scipy.sparse.linalg.eigsh`)
find the few eigenvalues you need without ever forming the full matrix.

### Scaling comparison

| Method | Time complexity | Memory |
|--------|-----------------|--------|
| Dense `eigh` | $O(N^3)$ | $O(N^2)$ |
| Sparse `eigsh` (k values) | $O(k \cdot N \cdot \text{nnz}/N)$ | $O(\text{nnz} + kN)$ |

### Quantum Connection
The difference between tractable and intractable quantum problems often comes
down to sparsity.  A 20-qubit Hamiltonian is a $10^6 \times 10^6$ matrix —
only feasible with sparse methods.

In [ ]:
# ── Dense vs sparse eigenvalue scaling ──────────────────────────────
sizes = [50, 100, 200, 500, 1000, 2000]
times_dense = []
times_sparse = []

for N in sizes:
    # Create a sparse tridiagonal matrix (like 1D Schrödinger)
    diag = 2.0 * np.ones(N)
    off = -1.0 * np.ones(N - 1)
    H = sparse.diags([off, diag, off], [-1, 0, 1], format='csr')

    # Dense eigenvalue decomposition (all eigenvalues)
    H_dense = H.toarray()
    t0 = time.perf_counter()
    evals_d = np.linalg.eigvalsh(H_dense)
    t_dense = time.perf_counter() - t0
    times_dense.append(t_dense)

    # Sparse eigensolver (only lowest 6 eigenvalues)
    t0 = time.perf_counter()
    evals_s, _ = eigsh(H, k=6, which='SA')
    t_sparse = time.perf_counter() - t0
    times_sparse.append(t_sparse)

    # Verify they agree
    max_diff = np.max(np.abs(np.sort(evals_s) - np.sort(evals_d[:6])))
    print(f"N={N:>5}: dense={t_dense*1000:>8.2f} ms, "
          f"sparse={t_sparse*1000:>8.2f} ms, "
          f"speedup={t_dense/t_sparse:>6.1f}x, "
          f"max diff={max_diff:.2e}")

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(sizes, [t*1000 for t in times_dense], 'o-',
          label='Dense (all eigenvalues)', markersize=8)
ax.loglog(sizes, [t*1000 for t in times_sparse], 's-',
          label='Sparse (6 eigenvalues)', markersize=8)

# Reference lines
s = np.array(sizes, dtype=float)
ax.loglog(s, 0.001 * (s/50)**3, 'k--', alpha=0.3, label=r'$O(N^3)$ reference')
ax.loglog(s, 0.05 * (s/50)**1.2, 'k:', alpha=0.3, label=r'$O(N^{1.2})$ reference')

ax.set_xlabel('Matrix size N')
ax.set_ylabel('Time (ms)')
ax.set_title('Dense vs Sparse Eigenvalue Solver Scaling')
ax.legend()
ax.grid(True, alpha=0.3, which='both')
plt.tight_layout()
plt.show()

## 7. Quantum Application: Particle in a Box (Numerical vs Exact)

Let us put all these tools together on a real quantum problem.

The 1D infinite square well has exact solutions:

$$\psi_n(x) = \sqrt{\frac{2}{L}} \sin\left(\frac{n\pi x}{L}\right), \quad
E_n = \frac{n^2 \pi^2 \hbar^2}{2mL^2}$$

We will solve the same problem numerically using finite differences and compare.

In [ ]:
# ── Quantum particle in a box: numerical vs analytical ──────────────
# Units: ℏ = m = 1, L = 1
L = 1.0
N_grid = 200
dx = L / (N_grid + 1)
x_grid = np.linspace(dx, L - dx, N_grid)  # interior points only

# Finite-difference Hamiltonian: H = -ℏ²/(2m) d²/dx²
# Second derivative: d²ψ/dx² ≈ (ψ_{i+1} - 2ψ_i + ψ_{i-1}) / dx²
coeff = 1.0 / (2.0 * dx**2)
H_box = sparse.diags(
    [-coeff * np.ones(N_grid - 1),
     2 * coeff * np.ones(N_grid),
     -coeff * np.ones(N_grid - 1)],
    [-1, 0, 1], format='csr'
)

# Solve for lowest 5 states
n_states = 5
energies, states = eigsh(H_box, k=n_states, which='SA')
idx = np.argsort(energies)
energies = energies[idx]
states = states[:, idx]

# Normalize eigenstates
for i in range(n_states):
    norm = np.sqrt(np.trapezoid(states[:, i]**2, x_grid))
    states[:, i] /= norm
    # Fix sign convention: positive at first peak
    if states[N_grid // (2*(i+1)), i] < 0:
        states[:, i] *= -1

# Exact solutions
def exact_psi(n, x):
    return np.sqrt(2/L) * np.sin(n * np.pi * x / L)

def exact_E(n):
    return (n * np.pi)**2 / 2.0

# Compare
print(f"{'n':>3} {'E_numerical':>14} {'E_exact':>14} {'Rel. Error':>14}")
print("-" * 48)
for i in range(n_states):
    n = i + 1
    E_ex = exact_E(n)
    rel_err = abs(energies[i] - E_ex) / E_ex
    print(f"{n:>3} {energies[i]:>14.8f} {E_ex:>14.8f} {rel_err:>14.2e}")

# Plot wavefunctions
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
x_fine = np.linspace(0, L, 500)

for i in range(n_states):
    n = i + 1
    offset = energies[i]
    scale = 0.3 * (energies[1] - energies[0])  # for visual separation

    # Numerical
    axes[0].plot(x_grid, states[:, i] * scale + offset, f'C{i}', linewidth=2)
    axes[0].axhline(offset, color=f'C{i}', linewidth=0.5, alpha=0.3)

    # Exact
    axes[1].plot(x_fine, exact_psi(n, x_fine) * scale + offset, f'C{i}',
                 linewidth=2, label=f'n={n}')
    axes[1].axhline(offset, color=f'C{i}', linewidth=0.5, alpha=0.3)

for ax, title in zip(axes, ['Numerical (finite difference)', 'Exact (analytical)']):
    ax.set_xlabel('x / L')
    ax.set_ylabel('Energy + ψ(x)')
    ax.set_title(title)
    ax.set_xlim(0, L)
    ax.grid(True, alpha=0.3)

axes[1].legend(loc='upper right')
plt.suptitle('Infinite Square Well: Numerical vs Exact', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## Summary: Scientific Computing Toolkit

| Tool | SciPy Function | Use Case |
|------|---------------|----------|
| Broadcasting | NumPy array ops | Efficient multi-dimensional computation |
| Sparse matrices | `scipy.sparse` | Large Hamiltonians, lattice models |
| Integration | `quad`, `dblquad`, Monte Carlo | Expectation values, normalization |
| ODE solving | `solve_ivp` (RK45, Radau, BDF) | Time evolution, Schrödinger equation |
| Optimization | `minimize`, `root_scalar`, `curve_fit` | Variational methods, energy levels |
| Eigenvalues | `eigsh`, `eigh` | Energy spectra, quantum states |

### Key Takeaway
These numerical tools are the **computational backbone** of quantum physics.
Every technique practiced here — from sparse eigensolvers to Monte Carlo
integration — maps directly onto quantum simulation tasks you will encounter
starting in Year 1.

---
*Notebook generated for the SIIEA Quantum Engineering curriculum.*
*License: CC BY-NC-SA 4.0 | Siiea Innovations, LLC*